# 2025大罷免沃草.json 對應處理程式
- version : v0.1


```markdown
## Cell 1: 載入套件與資料 (Updated)


資料格式與範例資料
[
  {
    "DocumentID": "BA2025-001",
    "Title": "2025 大罷免》為什麼要罷免傅崐萁？",
    "Source": "沃草",
    "Author": "何宇軒",
    "DatePublished": "2024-05-22",
    "DocumentType": "新聞報導",
    "Topic": ["罷免", "傅崐萁", "國會擴權", "親中", "花蓮"],
    "CoverageArea": "地方",
    "SpecificLocation": "花蓮縣",
    "TargetPoliticians": [
      {
        "TargetPoliticianID": "TP001",
        "TargetPoliticianName": "傅崐萁",
        "TargetPoliticianParty": "中國國民黨",
        "TargetPoliticianDistrict": "花蓮縣",
        "TargetPoliticianOffice": "立法委員",
        "ReasonForRecall": ["強勢領導作風", "親中立場", "推動國會擴權法案", "刪減國防預算", "會見中共統戰頭子", "對嗆退休教師"],
        "ReasonCategory": ["政治立場", "政策失當", "道德爭議"],
        "MentionFrequency": 30,
        "Sentiment": "負面",
        "RelatedLegislation": ["國會擴權法案", "選罷法", "憲法訴訟法", "財政收支劃分法"]
      }
    ],
    "RelatedPeople": [
      {
        "RelatedPersonName": "柯志恩",
        "RelatedPersonRole": "立法委員",
        "RelationshipToTarget": "同黨",
        "MentionFrequency": 3
      },
      {
        "RelatedPersonName": "黃捷",
        "RelatedPersonRole": "立法委員",
        "RelationshipToTarget": "對手",
        "MentionFrequency": 2
      },

    ],
    "Events": [
      {
        "EventName": "立法院通過財劃法",
        "EventDate": "2024-05-17",
        "EventType": "法案審查",
        "EventLocation": "立法院",
        "EventDescription": "立法院強勢通過《財劃法》，引發爭議。",
        "InvolvedPoliticianIDs": ["TP001"]
      },
      {
        "EventName": "傅崐萁率團訪中",
        "EventDate": "2024-04-26",
        "EventType": "政治活動",
        "EventLocation": "中國",
        "EventDescription": "傅崐萁於立法院開會期間率團訪中，會見王滬寧。",
        "InvolvedPoliticianIDs": ["TP001"]
      },
    ]
  },
]


In [1]:

import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# 指定 JSON 檔案名稱
json_file_path = "2025大罷免沃草.json"  # 請確保檔案 'recall_data.json' 與 Notebook 在同一個目錄，或提供完整路徑

# 從檔案讀取 JSON 資料
try:
    with open(json_file_path, 'r', encoding='utf-8') as f: # 加上 encoding='utf-8' 處理中文
        loaded_json_data = json.load(f)
    print(f"成功從 '{json_file_path}' 讀取資料！")
except FileNotFoundError:
    print(f"錯誤：找不到檔案 '{json_file_path}'。請確認檔案路徑是否正確，並將 JSON 資料儲存為 '{json_file_path}' 檔案。")
    loaded_json_data = [] # 若檔案不存在，載入空資料，避免後續程式碼錯誤
except json.JSONDecodeError:
    print(f"錯誤：檔案 '{json_file_path}' JSON 格式不正確。請檢查檔案內容是否為有效的 JSON 格式。")
    loaded_json_data = []

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
成功從 '2025大罷免沃草.json' 讀取資料！


## Cell 2: 建立 `RecallDataAnalyzer` 物件與資料轉化

In [3]:
class RecallDataAnalyzer:
    def __init__(self, json_data):
        self.raw_data = json_data # 原始 JSON 資料

        self.politician_parties = self._get_politician_parties()
        self.recall_reasons = self._get_recall_reasons()
        self.reason_categories = self._get_reason_categories()
        self.document_types = self._get_document_types()
        self.coverage_areas = self._get_coverage_areas()
        self.specific_locations = self._get_specific_locations()
        self.topics = self._get_topics()
        self.sentiments = self._get_sentiments()
        self.event_types = self._get_event_types()
        self.related_people_roles = self._get_related_people_roles()
        self.reason_keywords_counts = self._get_reason_keywords_counts()
        self.target_politicians = self._get_target_politicians()  # 新增變數


    def _get_politician_parties(self):
        parties = [tp['TargetPoliticianParty'] for doc in self.raw_data for tp in doc.get('TargetPoliticians', [])]
        return Counter(parties)

    def _get_recall_reasons(self):
        reasons_dict = {}
        for doc in self.raw_data:
            for tp in doc.get('TargetPoliticians', []):
                reasons_dict[tp['TargetPoliticianName']] = tp['ReasonForRecall']
        return reasons_dict

    def _get_reason_categories(self):
        categories = [cat for doc in self.raw_data for tp in doc.get('TargetPoliticians', []) for cat in tp.get('ReasonCategory', [])]
        return Counter(categories)

    def _get_document_types(self):
        doc_types = [doc['DocumentType'] for doc in self.raw_data]
        return Counter(doc_types)

    def _get_coverage_areas(self):
        areas = [doc['CoverageArea'] for doc in self.raw_data]
        return Counter(areas)

    def _get_specific_locations(self):
        locations = [doc['SpecificLocation'] for doc in self.raw_data if doc.get('SpecificLocation')] # 處理 None 值
        return Counter(locations)

    def _get_topics(self):
        topics = [topic for doc in self.raw_data for topic in doc['Topic']]
        return Counter(topics)

    def _get_sentiments(self):
        sentiments = [tp['Sentiment'] for doc in self.raw_data for tp in doc.get('TargetPoliticians', []) if tp.get('Sentiment')] # 處理 None 值
        return Counter(sentiments)

    def _get_event_types(self):
        event_types = [event['EventType'] for doc in self.raw_data for event in doc.get('Events', [])]
        return Counter(event_types)

    def _get_related_people_roles(self):
        roles = [person['RelatedPersonRole'] for doc in self.raw_data for person in doc.get('RelatedPeople', [])]
        return Counter(roles)

    def _get_reason_keywords_counts(self, top_n=10):
        all_reasons = [reason for doc in self.raw_data for tp in doc.get('TargetPoliticians', []) for reason in tp['ReasonForRecall']]
        reason_words = []
        for reason in all_reasons:
            reason_words.extend(reason.split())
        return Counter(reason_words).most_common(top_n)

    
    def _get_target_politicians(self):
        return {tp['TargetPoliticianName']: tp for doc in self.raw_data for tp in doc.get('TargetPoliticians', [])}

## Cell 3: 基本資料檢視


In [4]:

analyzer = RecallDataAnalyzer(loaded_json_data)
print(f"文件數量: {len(analyzer.raw_data)}")
print(f"被罷免政黨數量: {len(analyzer.politician_parties)}") # 使用實例變數
print("\n第一個文件的 Keys:", analyzer.raw_data[0].keys() if analyzer.raw_data else "No data loaded") # 使用實例變數

文件數量: 27
被罷免政黨數量: 2

第一個文件的 Keys: dict_keys(['DocumentID', 'Title', 'Source', 'Author', 'DatePublished', 'DocumentType', 'Topic', 'CoverageArea', 'SpecificLocation', 'TargetPoliticians', 'RelatedPeople', 'Events'])


## Cell 4： 變數檢視

In [5]:

print(analyzer.politician_parties) 
print(analyzer.recall_reasons )
print(analyzer.reason_categories )
print(analyzer.document_types )
print(analyzer.coverage_areas )
print(analyzer.specific_locations) 
print(analyzer.topics )
print(analyzer.sentiments )
print(analyzer.event_types )
print(analyzer.related_people_roles )
print(analyzer.reason_keywords_counts) 
print(analyzer.target_politicians)



Counter({'中國國民黨': 22, '無黨籍 (國民黨團)': 1})
{'傅崐萁': ['強勢領導作風', '親中立場', '推動國會擴權法案', '刪減國防預算', '會見中共統戰頭子', '對嗆退休教師'], '王鴻薇': ['刪減消防員預算', '參與傅崐萁統戰團', '凍結中選會預算', '親中言論', '肉搜公民'], '徐巧芯': ['家族涉詐騙', '違停爭議', '奢侈品爭議', '比中指', '罵髒話', '承認暴力行為', '凍結潛艦預算', '親中言論'], '葉元之': ['機密預算直播洩密', '慣老闆爭議', '刪減預算', '依賴空戰', '投機性格'], '牛煦庭': ['試用期8成薪', '反同爭議', '刪減性平預算', '暴打立委', '臉書罵網友', '行政干預罷免'], '李彥秀': ['美國豪宅未申報', '綠卡/公民身份質疑', '親中疑美', '削弱國防預算'], '徐欣瑩': ['修惡選罷法', '訪中', '凍結客委會預算', '刪減性平預算', '宗教色彩濃厚', '議事瑕疵', '跑去洗頭'], '邱鎮軍': ['推沈伯洋致腦震盪', '巴頭許智傑', '黑道背景', '賭博電玩爭議', '與統促黨關係密切', '暴力傾向'], '游顥': ['縮短中配入籍', '替救國團解套', '訪中', '政見跳票', '澎湖設服務處', '財劃法損害南投利益'], '羅明才': ['父親羅福助黑道背景', '大香山招待所爭議', '提倡發紅包民粹政策', '提倡兩岸和平協議', '參與傅崐萁訪中團', '刪減轉型正義預算'], '顏寬恒': ['家族黑金勢力', '貪污罪判刑', '土地爭議', '質詢率低', '配合黨團通過惡法'], '廖偉翔': ['收割中捷藍線政績', '質詢出包', '打人', '氣爆現場直播', '專業度受質疑', '魁儡網紅'], '呂玉玲': ['凍結國防預算', '刪減客家預算', '軍系立委形象不符', '賄選爭議', '黨團橡皮圖章'], '馬文君': ['阻撓潛艦國造', '洩密疑雲', '農地豪宅爭議', '提案刪減陸委會預算', '支持中配入籍'], '陳超明': ['SOGO收賄案', '烏龍言論', '石虎爭議', '支持賄選言論', '歧視同性戀言論', '口